In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage
from scipy.signal import find_peaks
from scipy.signal import medfilt
from scipy.signal import savgol_filter

In [2]:
def moving_average(x, window=3):
    box = np.ones(window)/window
    smoothed = np.convolve(x, box, mode="same")
    return smoothed

In [3]:
def normalize_df(input_df):
    for col in input_df.columns:
        input_df[col] = input_df[col]/input_df[col].max()
    return input_df        

In [4]:
def horizontal_sum(df):
    
    combined_signal = []
    
    signal_len = len(df.iloc[:, 0])
    
    for index in range(0, signal_len): 
        sum_horiz = 0
        for signal in df.columns:
            sum_horiz += df[signal][index]
        
        combined_signal.append(sum_horiz)
        
    return combined_signal
        

In [5]:
def compute_periodogram(input_signal):
    signal = np.array(input_signal) #np.array(input_signal.apply(lambda x: float(x)))
    pgram_python = (2.0/float(signal.size)) * np.power(np.fft.fft(signal), 2)
    indices = list(range(0, int(len(pgram_python)/2 + 1)))
    pgram_python = pgram_python[indices]
    return np.array(pgram_python.real)

In [6]:
def filter_data(input_signal):
    signal = []
    
    for val in input_signal:
        if(not np.isnan(val)):
            signal.append(val)
            
    signal = np.array(signal)
    return signal

In [7]:
def gaussian_filter(input_signal):
    sigma = 2 #play around with this value to influence the degree of "smoothness"
    signal = ndimage.gaussian_filter1d(input_signal, sigma)
    return signal

In [8]:
def determine_squats(periodogram):
    
    periodogram = periodogram
    
    periodogram = list(periodogram)
    
    frames = len(periodogram)
        
    peaks = find_peaks(periodogram)
    
    peaks = peaks[0]
        
    max_peak = 0
    max_power = 0
    
    for peak in peaks:
        if(periodogram[peak] > max_power):
            max_peak = peak
            max_power = periodogram[peak]

    max_frame = max_peak
    
    max_freq = max_frame/frames
    
    period = 1/max_freq
    
    num_squats = frames/period
        
    return num_squats
    

In [21]:
def complete_pipeline(filepath):
    df = pd.read_csv(filepath, sep='\s*,\s*', encoding='ascii', engine='python', skiprows=[1])
    df.columns = map(lambda x: x.strip(), list(df.columns)) #remove preceding " " and trailing " " from column names
    
    df = df.iloc[:, [17, 19, 21]]#df.iloc[:, 15:]# #just get the interpolated values from the CSV [17, 19, 21]
    
    df = normalize_df(df) #normalized signals before combining them
    combined_signal = horizontal_sum(df) #combine signals

    combined_signal = filter_data(combined_signal) #remove NaN

    combined_signal = medfilt(combined_signal, kernel_size=7) #use running median to get rid of outliers
    #combined_signal = gaussian_filter(combined_signal)
    
    
    pgram = compute_periodogram(combined_signal)
    pgram = pgram[1:int(len(pgram)/2)]
    
    squats = determine_squats(pgram)
    
    return squats

In [22]:
indexes = [[1, 7], [2, 7], [3, 7], [5, 7], [6, 10], [7, 10], [8, 10], [9, 11], [10, 10], [11, 10], [12, 10], [13, 10], [14, 8], [15, 20], [16, 8.5], [17, 8], [18, 9], [19, 8.5], [20, 9], [21, 9], [22, 9], [23, 9]]
running_error = 0
for trial in indexes:
    index = trial[0]
    true_val = trial[1]
    pred_val = complete_pipeline("/Users/sreeharirammohan/Downloads/STS Trials/trial_" + str(index) + ".csv")
    percent_error = abs(true_val - pred_val)/true_val
    running_error += percent_error
    print("Trial " + str(index) + ": " + str(round(percent_error, 2)) + " -> " + str(complete_pipeline("/Users/sreeharirammohan/Downloads/STS Trials/trial_" + str(index) + ".csv")) + " vs. " + str(true_val))
print(100*running_error/len(indexes))

Trial 1: 1.0 -> 14.0 vs. 7
Trial 2: 0.29 -> 5.0 vs. 7
Trial 3: 0.86 -> 13.0 vs. 7
Trial 5: 0.29 -> 5.0 vs. 7
Trial 6: 0.2 -> 8.0 vs. 10
Trial 7: 0.1 -> 9.0 vs. 10
Trial 8: 0.8 -> 18.0 vs. 10
Trial 9: 0.82 -> 2.0 vs. 11
Trial 10: 0.1 -> 9.0 vs. 10
Trial 11: 0.1 -> 9.0 vs. 10
Trial 12: 0.1 -> 9.0 vs. 10
Trial 13: 0.3 -> 13.000000000000002 vs. 10
Trial 14: 1.13 -> 17.000000000000004 vs. 8
Trial 15: 0.75 -> 5.000000000000001 vs. 20
Trial 16: 1.47 -> 21.0 vs. 8.5
Trial 17: 0.12 -> 7.000000000000001 vs. 8
Trial 18: 0.33 -> 6.000000000000001 vs. 9
Trial 19: 0.88 -> 1.0 vs. 8.5
Trial 20: 0.11 -> 8.0 vs. 9
Trial 21: 0.0 -> 9.0 vs. 9
Trial 22: 0.89 -> 17.0 vs. 9
Trial 23: 0.11 -> 8.0 vs. 9
48.83699485303764
